In [2]:
# Import Libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Import data
df = pd.read_csv('Google_Stock_Price_Train.csv')

In [4]:
# read data
df.sample(10)

,Date,Open,High,Low,Close,Volume
1170,8/26/2016,769.00,776.08,765.85,769.54,"1,166,700"
214,11/8/2012,333.85,334.49,324.40,650.5,"5,213,300"
350,5/28/2013,440.10,444.40,438.56,878.86,"4,531,900"
170,9/5/2012,338.73,341.97,338.30,678.86,"3,429,100"
725,11/19/2014,533.54,536.77,528.63,535.52,"1,392,200"
1045,3/1/2016,703.62,718.81,699.77,718.81,"2,151,400"
1076,4/14/2016,754.01,757.31,752.71,753.2,"1,135,300"
1067,4/1/2016,738.60,750.34,737.00,749.91,"1,576,700"
737,12/8/2014,525.69,529.55,522.36,525.54,"2,329,300"
450,10/17/2013,444.83,446.78,441.21,886.36,"8,545,000"


In [5]:
# Understanding data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1258 non-null   object 
 1   Open    1258 non-null   float64
 2   High    1258 non-null   float64
 3   Low     1258 non-null   float64
 4   Close   1258 non-null   object 
 5   Volume  1258 non-null   object 
dtypes: float64(3), object(3)
memory usage: 59.1+ KB


In [6]:
df.describe()

,Open,High,Low
count,1258.000000,1258.000000,1258.000000
mean,533.709833,537.880223,529.007409
std,151.904442,153.008811,150.552807
min,279.120000,281.210000,277.220000
25%,404.115000,406.765000,401.765000
50%,537.470000,540.750000,532.990000
75%,654.922500,662.587500,644.800000
max,816.680000,816.680000,805.140000


In [16]:
# Select Features
y = df['Open']

# Scaling data
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()
y = scale.fit_transform(y.values.reshape(-1,1))
y

array([[0.08581368],
       [0.09701243],
       [0.09433366],
       ...,
       [0.95725128],
       [0.93796041],
       [0.93688146]])

In [21]:
# creating a data structure with 60 timestips and 1 output
X_train = []
y_train = []
for i in range(60,len(y)):
    X_train.append(y[i-60:i,0])
    y_train.append(y[i,0])

In [23]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [25]:
X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],1))

In [26]:
X_train.shape

(1198, 60, 1)

In [27]:
## Create model

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [29]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

In [30]:
regressor.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 50)            10400     
                                                                 
 dropout (Dropout)           (None, 60, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_1 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_2 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_2 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_3 (LSTM)               (None, 50)                2

In [31]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [32]:
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
38/38 [==============================] - 10s 83ms/step - loss: 0.0331
Epoch 2/100
38/38 [==============================] - 3s 86ms/step - loss: 0.0063
Epoch 3/100
38/38 [==============================] - 3s 84ms/step - loss: 0.0056
Epoch 4/100
38/38 [==============================] - 3s 88ms/step - loss: 0.0054
Epoch 5/100
38/38 [==============================] - 3s 85ms/step - loss: 0.0052
Epoch 6/100
38/38 [==============================] - 3s 86ms/step - loss: 0.0052
Epoch 7/100
38/38 [==============================] - 3s 83ms/step - loss: 0.0043
Epoch 8/100
38/38 [==============================] - 3s 85ms/step - loss: 0.0045
Epoch 9/100
38/38 [==============================] - 4s 95ms/step - loss: 0.0041
Epoch 10/100
38/38 [==============================] - 3s 84ms/step - loss: 0.0048
Epoch 11/100
38/38 [==============================] - 3s 85ms/step - loss: 0.0045
Epoch 12/100
38/38 [==============================] - 3s 83ms/step - loss: 0.0037
Epoch 13/100
38/38 [====

38/38 [==============================] - 3s 91ms/step - loss: 0.0015


In [36]:
# Model Evaluation
dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
dataset_test.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2017,778.81,789.63,775.80,786.14,"1,657,300"
1,1/4/2017,788.36,791.34,783.16,786.90,"1,073,000"
2,1/5/2017,786.08,794.48,785.02,794.02,"1,335,200"
3,1/6/2017,795.26,807.90,792.20,806.15,"1,640,200"
4,1/9/2017,806.40,809.97,802.83,806.65,"1,272,400"


In [38]:
dataset_total = pd.concat((df['Open'], dataset_test['Open']), axis = 0)
dataset_total.head()

0    325.25
1    331.27
2    329.83
3    328.34
4    322.04
Name: Open, dtype: float64

In [39]:
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs

array([779.  , 779.66, 777.71, 786.66, 783.76, 781.22, 781.65, 779.8 ,
       787.85, 798.24, 803.3 , 795.  , 804.9 , 816.68, 806.34, 801.  ,
       808.35, 795.47, 782.89, 778.2 , 767.25, 750.66, 774.5 , 783.4 ,
       779.94, 791.17, 756.54, 755.6 , 746.97, 755.2 , 766.92, 771.37,
       762.61, 772.63, 767.73, 764.26, 760.  , 771.53, 770.07, 757.44,
       744.59, 757.71, 764.73, 761.  , 772.48, 780.  , 785.04, 793.9 ,
       797.4 , 797.34, 800.4 , 790.22, 796.76, 795.84, 792.36, 790.9 ,
       790.68, 793.7 , 783.33, 782.75, 778.81, 788.36, 786.08, 795.26,
       806.4 , 807.86, 805.  , 807.14, 807.48, 807.08, 805.81, 805.12,
       806.91, 807.25, 822.3 , 829.62, 837.81, 834.71, 814.66, 796.86])

In [40]:
inputs.shape

(80,)

In [42]:
inputs = inputs.reshape(-1,1)
inputs = scale.transform(inputs)

In [43]:
inputs.shape

(80, 1)

In [44]:
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])

In [45]:
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [46]:
X_test

array([[[0.9299055 ],
        [0.93113327],
        [0.92750577],
        ...,
        [0.95725128],
        [0.93796041],
        [0.93688146]],

       [[0.93113327],
        [0.92750577],
        [0.94415507],
        ...,
        [0.93796041],
        [0.93688146],
        [0.92955205]],

       [[0.92750577],
        [0.94415507],
        [0.93876032],
        ...,
        [0.93688146],
        [0.92955205],
        [0.94731751]],

       ...,

       [[0.96054394],
        [0.9371419 ],
        [0.92841729],
        ...,
        [1.01045465],
        [1.02407173],
        [1.03930724]],

       [[0.9371419 ],
        [0.92841729],
        [0.90804747],
        ...,
        [1.02407173],
        [1.03930724],
        [1.03354044]],

       [[0.92841729],
        [0.90804747],
        [0.8771858 ],
        ...,
        [1.03930724],
        [1.03354044],
        [0.99624228]]])

In [47]:
X_test.shape

(20, 60, 1)

In [48]:
# Evaluating Model

In [49]:
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price

1/1 [==============================] - 1s 1s/step


array([[0.96885216],
       [0.96388876],
       [0.9638201 ],
       [0.96560466],
       [0.97098243],
       [0.9810802 ],
       [0.99081135],
       [0.9956192 ],
       [0.9974258 ],
       [0.9979067 ],
       [0.99789226],
       [0.99749315],
       [0.99707353],
       [0.9978365 ],
       [0.9993658 ],
       [1.0072389 ],
       [1.0191566 ],
       [1.0323715 ],
       [1.040354  ],
       [1.0347624 ]], dtype=float32)

In [51]:
predicted_stock_price = scale.inverse_transform(predicted_stock_price)
predicted_stock_price

array([[799.93616],
       [797.268  ],
       [797.2311 ],
       [798.1904 ],
       [801.0813 ],
       [806.50946],
       [811.74054],
       [814.325  ],
       [815.2962 ],
       [815.5547 ],
       [815.54694],
       [815.3324 ],
       [815.1068 ],
       [815.51697],
       [816.33905],
       [820.5713 ],
       [826.9778 ],
       [834.0816 ],
       [838.3727 ],
       [835.3668 ]], dtype=float32)

In [56]:
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()

NameError: name 'real_stock_price' is not defined